In [6]:
import dgl
import copy
import torch
import random
import numpy as np
import pandas as pd
from model import GCN_NET
from sklearn.metrics import accuracy_score


def setup_seed(seed):
    """
    fix the random seed
    :param seed: the random seed
    """
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    np.random.seed(seed)
    random.seed(seed)
    torch.manual_seed(seed)
    torch.backends.cudnn.benchmark = False
    torch.backends.cudnn.deterministic = True
    return None

setup_seed(0)

# attribute
name2id = {}
id2name = {}
x = pd.read_csv("data/features.csv")

for i in range(len(x)):
    name = x.loc[i][0]
    id2name[i] = name
    name2id[name] = i

x = x.iloc[:, 1:]
x = x.to_numpy()

# attribute
# adj = pd.read_excel("adj.xlsx")
adj = pd.read_csv("data/Network.csv")

max_id = x.shape[0]
edges = []
for i in range(len(adj)):
    start = adj.loc[i][0]
    mid = adj.loc[i][1]
    end = adj.loc[i][2]
    # if start != "NA":
    #     try:
    #         name2id[start]
    #     except:
    #         name2id[start] = max_id
    #         id2name[max_id] = start
    #         max_id += 1
    #
    # if mid != "NA":
    #     try:
    #         name2id[mid]
    #     except:
    #         name2id[mid] = max_id
    #         id2name[max_id] = mid
    #         max_id += 1
    #
    # if end != "NA":
    #     try:
    #         name2id[end]
    #     except:
    #         name2id[end] = max_id
    #         id2name[max_id] = end
    #         max_id += 1

    if 1:
        try:
            edges.append([name2id[start], name2id[mid]])
        except:
            pass
        try:
            edges.append([name2id[mid], name2id[start]])
        except:
            pass
    if 1:
        try:
            edges.append([name2id[mid], name2id[end]])
        except:
            pass
        try:
            edges.append([name2id[end], name2id[mid]])
        except:
            pass

label = pd.read_csv("data/label.csv")

y = np.zeros((x.shape[0], ))
for i in range(len(label["node"])):
    try:
        index = name2id[label["node"].iloc[i]]
        y[index] = label["label"].iloc[i]
    except:
        pass

y = torch.tensor(y).float()

# x = np.concatenate([x, np.zeros((len(name2id)-x.shape[0], 5))], axis=0)

x = torch.tensor(x).float()
# print(x.shape)
# print(len(id2name))
# print(len(name2id))
g = dgl.graph(edges)
# print(g.nodes().shape)

# print(x.shape)
# print(g)

# GCN
criterion = torch.nn.BCELoss()
net = GCN_NET()
optimizer = torch.optim.Adam(net.parameters(), lr=1e-1)

idx = list(range(x.shape[0]))
random.shuffle(idx)
idx = np.array(idx)

C:\Users\ASUS\AppData\Local\Temp\ipykernel_27976\2426522859.py:34: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  name = x.loc[i][0]
C:\Users\ASUS\AppData\Local\Temp\ipykernel_27976\2426522859.py:48: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  start = adj.loc[i][0]
C:\Users\ASUS\AppData\Local\Temp\ipykernel_27976\2426522859.py:49: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  mid = adj.loc[i][1]
C:\Users\ASUS\AppData\Local\Temp\ipy

In [7]:
print(len(label),len(x))

492 493


In [8]:
logits = net(g, x)

In [9]:
from sklearn.model_selection import KFold
import numpy as np
kf = KFold(n_splits=5,shuffle=False)  # 初始化KFold

In [10]:
for train_id, test_id in kf.split(x):
    #print('train_index:%s , test_index: %s ' %(train_id, test_id))
    best_acc = 0
    best_model = None
    threshold = 0.2

    for epoch in range(100):
        logits = net(g, x)
        loss = criterion(logits.squeeze(dim=-1)[train_id], y[train_id])
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        logits = net(g, x)
        # print(logits)
        y_pred = (logits >= threshold).int().squeeze(dim=-1)
        acc = accuracy_score(y[test_id], y_pred[test_id])
        #print(acc)
        if best_acc < acc:
            best_acc = acc
            best_model = copy.deepcopy(net)

    logits = best_model(g, x)
    y_pred = (logits >= threshold).int().squeeze(dim=-1)

    logits_list = logits[test_id][:, 0].data
    index = sorted(range(len(logits_list)), key=lambda k: logits_list[k], reverse=True)
    #print(test_id[index])


    acc = accuracy_score(y[test_id], y_pred[test_id])
    print(acc)


    from sklearn.metrics import confusion_matrix
    cm = confusion_matrix(y[test_id], y_pred[test_id])
    #print(cm)

    logits = logits[test_id]

    acc = accuracy_score(y[test_id], y_pred[test_id])
    print("final", acc)

    import seaborn as sns
    import pandas as pd
    import matplotlib.pyplot as plt
    from sklearn.metrics import confusion_matrix
    from sklearn.metrics import accuracy_score, precision_score, f1_score, recall_score, classification_report

    y_true = y[test_id]
    y_pred = y_pred[test_id]
    # 1.计算混淆矩阵
    cm = confusion_matrix(y_true, y_pred)
    conf_matrix = pd.DataFrame(cm, index=['0', '1'], columns=['0', '1'])  # 数据有5个类别

    from sklearn import metrics
    fpr, tpr, thresholds = metrics.roc_curve(y_true, logits.detach().numpy())
    auc = metrics.auc(fpr, tpr)
    #
    # # 2.计算accuracy
    # print('accuracy_score', accuracy_score(y_true, y_pred))
    #
    # # 3.计算多分类的precision、recall、f1-score分数
    print("final", acc)
    print('precision', precision_score(y_true, y_pred))
    print('recall', recall_score(y_true, y_pred))
    print('f1-score', f1_score(y_true, y_pred))
    print('auc', auc)
    print('###')

    

0.6464646464646465
final 0.6464646464646465
final 0.6464646464646465
precision 0.25925925925925924
recall 0.3181818181818182
f1-score 0.2857142857142857
auc 0.6056670602125148
###
0.7676767676767676
final 0.7676767676767676
final 0.7676767676767676
precision 0.7789473684210526
recall 0.9736842105263158
f1-score 0.8654970760233919
auc 0.677345537757437
###
0.5252525252525253
final 0.5252525252525253
final 0.5252525252525253
precision 0.5154639175257731
recall 1.0
f1-score 0.6802721088435374
auc 0.6591836734693879
###
0.24489795918367346
final 0.24489795918367346
final 0.24489795918367346
precision 0.24489795918367346
recall 1.0
f1-score 0.39344262295081966
auc 0.6117680180180181
###
0.3469387755102041
final 0.3469387755102041
final 0.3469387755102041
precision 0.3333333333333333
recall 1.0
f1-score 0.5
auc 0.5958806818181818
###
